In [129]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
# from datatools.analyzer import clean_text
from utterance.error_tools import *

In [130]:
class Utterance:
    utt_level = ["Wrong information", "Semantic error", "Uninterpretable", "Grammatical error"]
    def __init__(self, did, sp, utt, errors, type_) -> None:
        self.did = did
        self.sp = sp
        self.utt = utt
        self.errors = errors
        self.type_ = type_
    
    def __str__(self):
        return "{0}: {1}".format(self.sp, self.utt)

    def is_system(self):
        return True if self.sp=="S" else False
    
    def is_error_included(self, error):
        # Null 対応
        if not self.errors:
            return False
        return error in self.errors
    
    def is_exist_error(self):
        return True if self.errors else False
    
    def is_type_included(self, type_):
        return type_ in self.type_
    
    def is_utt_level_error(self):
        for e in Utterance.utt_level:
            if self.is_error_included(e):
                return True
        return False
    

In [131]:
import json
from pathlib import Path
def read_conv(path:str, datalist:list):
    convs = []
    for p in datalist:
        datapath = Path(path + p + '/')
        for file_ in datapath.glob("*.json"):
            conv = []
            with open(file_, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    sp = t["speaker"]
                    utt = t["utterance"]
                    errors = t["error_category"]
                    type_ = t["type"]
                    one = Utterance(did, sp, utt, errors, type_)
                    conv.append(one)
            convs.append(conv)
    return convs      

In [132]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']


In [133]:
convs = read_conv(path, datalist)

In [134]:
utt_level = ["Wrong information", "Semantic error", "Uninterpretable", "Grammatical error"]
def extract_response(convs, type_):
    type_utt = []
    for conv in convs:
        for i, ut in enumerate(conv):
            if i == len(conv)-1:
                continue
            # システムの質問に対する応答
            if ut.is_system() and ut.is_type_included(type_) and not ut.is_exist_error():
                if not ut.is_utt_level_error():
                    if conv[i+1].is_type_included(type_):
                        print("後ろ+前", conv[i+1], conv[i+1].did)
                    else:
                        print("後ろ", conv[i+1])

In [135]:
extract_response(convs, "質問")

後ろ U: 1000円ぐらいかな？
後ろ U: はい。天丼を食べました。
後ろ U: たまに家族で外食に行くよ。
後ろ U: 毎年、家族で西新井大師に初詣に行くのが恒例です。
後ろ U: ボーっとするのが好きなので、嫌いではないですよ
後ろ U: 雨がやみました
後ろ U: 九州に旅行に行きます。
後ろ+前 U: チョコレートです。あなたの好きな食べ物はなんですか？ 1470624084
後ろ U: 出ていますよ。
後ろ U: そうですね。国内で死者が出ています。
後ろ U: 友達と行った。
後ろ U: 残業はごくたまにありますね。
後ろ U: 朝の海も気持ちよさそうですね。
後ろ U: 聞いたことがないですね
後ろ U: これから買い物に行って決めます。
後ろ U: 5人です。3姉妹なんです。
後ろ U: 仲良いですよ
後ろ U: SNSはしたことがないんです。
後ろ+前 U: やはり都合が悪いですか？ 1470624599
後ろ U: 夏の晴れた日の海は最高ですね
後ろ U: そうですね。無自覚のうちになってしまうこともあると聞きますから。喉が渇いていなくても水分補給はこまめにしたほうがいいみたいですよ。
後ろ U: 好きです。
後ろ U: 暑いね
後ろ U: もちろん
後ろ U: 昔、猫を飼っていました
後ろ U: 今はインターネットでの買い物が多いです。
後ろ U: ちょっとめまいがするけど、大丈夫。
後ろ U: あまり酷くはありません。
後ろ U: 紛争やテロなどが無くなることはありませんね。
後ろ U: たまにだね
後ろ U: わからないけど、おいしいよね。
後ろ U: どちらも行ったことがないので分かりませんが、すずらんは好きな花です。
後ろ U: 菅田将暉やね
後ろ U: 髭面が好きかな
後ろ U: あるけど、私はニコニコ生放送をやっていたよ。
後ろ U: 見ていないのでわからないです
後ろ U: 田舎なので田んぼ道を歩いてほしいですね。
後ろ U: 私は名古屋の方が関心ありますね
後ろ U: いえ、知りませんでした。
後ろ U: 竹田城跡です
後ろ+前 U: どちらも知らない人です。どんな職業の人なのですか？ 1503630791
後ろ U: ニュージーランドかな。
後ろ U: 私も飲んだことないから飲みたいな。
後ろ U: だったらジーコさんにつ

In [136]:
# 